In [21]:
from data import CocoDatasetManager
from collections import Counter

In [2]:
# Example usage
coco = CocoDatasetManager()

# Later, load the saved datasets
loaded_train_dataset = coco.load_saved_dataset("/workspaces/anaconda/src/data/train")
loaded_val_dataset = coco.load_saved_dataset("/workspaces/anaconda/src/data/test")


Importing samples...
 100% |███████████████████| 50/50 [7.5ms elapsed, 0s remaining, 6.7K samples/s]       
Import complete
Importing samples...
 100% |███████████████████| 50/50 [7.9ms elapsed, 0s remaining, 6.4K samples/s]       
Import complete


In [41]:
def dominant_label_box_size(detections):
    max_area = 0
    dominant_class = None

    for detection in detections:
        x_min, y_min, width, height = detection.bounding_box
        area = width * height
        if area > max_area:
            max_area = area
            dominant_class = detection.label
    return dominant_class


In [26]:
def dominant_majority_class(detections):
    labels = [detection.label for detection in detections]
    most_common_label = Counter(labels).most_common(1)[0][0]
    return most_common_label


In [44]:
i=10
for sample in loaded_val_dataset:
    detection =sample.detections.detections
    dominant_bounding=dominant_label_box_size(detection)
    dominant_majority=dominant_majority_class(detection)
    print(f"file {sample.filepath[-10:]} -- dominant_bounding_box: {dominant_bounding} --dominant_majority: {dominant_majority}")
    i=i-1
    if i==0:
        break


file 005037.jpg -- dominant_bounding_box: bus --dominant_majority: person
file 006040.jpg -- dominant_bounding_box: train --dominant_majority: person
file 007088.jpg -- dominant_bounding_box: person --dominant_majority: umbrella
file 009891.jpg -- dominant_bounding_box: car --dominant_majority: car
file 011197.jpg -- dominant_bounding_box: person --dominant_majority: traffic light
file 017207.jpg -- dominant_bounding_box: bus --dominant_majority: car
file 017627.jpg -- dominant_bounding_box: bus --dominant_majority: car
file 018837.jpg -- dominant_bounding_box: truck --dominant_majority: person
file 021839.jpg -- dominant_bounding_box: person --dominant_majority: person
file 025393.jpg -- dominant_bounding_box: person --dominant_majority: car
